# Using Python to Query MySQL

This notebook demonstrates using a couple of different database connectivity libraries to connect to and query a MySQL database.

- PyMySQL library
- MySQL's Native mysql.connector library
- SqlAlchemy library

## 1.0. Prerequisites

### 1.1. First, you must install the libaries into your python environment by executing the following commands in a Terminal window
- python3 -m pip install PyMySQL
- python3 -m pip install mysql.connector
- python3 -m pip install sqlalchemy

### 1.2. Next, as with all Jupyter Notebooks, you need to Import the libaries that you'll be working with in the notebook,

In [1]:
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import pandas as pd
import matplotlib.pyplot as plt

### 1.3. And then, Assign Connection Variables for the MySQL Server & Database with which You'll be Working

In [2]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "root"
pwd = "Tysonpookie1230!"
db_name = "northwind"

## 2.0. Using the PyMySQL Library

### 2.1. Using a Cursor to Iterate the Rows Returned

In [ ]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute('SELECT * FROM products;')
    
    for row in cursor.fetchmany(size=5):
        print(row)
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")
    
conn.close()

In [ ]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor(pymysql.cursors.DictCursor)

try:
    cursor.execute('SELECT * FROM products ORDER BY list_price DESC LIMIT 5;')
    
    for row in cursor.fetchall():
        print(row)
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")
    
conn.close()

### 2.2. Using the Pandas read_sql() Method to Return a DataFrame

In [ ]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)

df = pd.read_sql("SELECT * FROM products ORDER BY list_price DESC;", conn)

conn.close()
df.head()

## 3.0. Using the MySQL Native Connection Library

### 3.1. Using a Cursor to Iterate the Rows Returned

In [3]:
lbound = 15.00
ubound = 20.00

sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN %s AND %s
    ORDER BY list_price DESC;
"""

In [4]:
conn = mysql.connector.connect(user=user_id, password=pwd,
                               host=host_ip, database=db_name, 
                               auth_plugin='mysql_native_password')

In [5]:
cursor = conn.cursor()

try:
    cursor.execute(sql_query, (lbound, ubound))
    
    for (product_id, product_name, list_price) in cursor:
        print("{}, {} was sold for {}".format(product_id, product_name, list_price))
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")   

57, Northwind Traders Ravioli was sold for 19.5000
40, Northwind Traders Crab Meat was sold for 18.4000
1, Northwind Traders Chai was sold for 18.0000
66, Northwind Traders Tomato Sauce was sold for 17.0000
86, Northwind Traders Cake Mix was sold for 15.9900


In [6]:
cursor = conn.cursor()

try:
    cursor.execute(sql_query, (lbound, ubound))
    
    row = cursor.fetchone() 
    while row:
        print(row)
        row = cursor.fetchone()
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")

(57, 'Northwind Traders Ravioli', Decimal('19.5000'))
(40, 'Northwind Traders Crab Meat', Decimal('18.4000'))
(1, 'Northwind Traders Chai', Decimal('18.0000'))
(66, 'Northwind Traders Tomato Sauce', Decimal('17.0000'))
(86, 'Northwind Traders Cake Mix', Decimal('15.9900'))


In [7]:
conn.close()

### 3.2 Using Pandas read_sql() Method to Return a DataFrame

In [8]:
lbound = 15.00
ubound = 20.00

sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN {0} AND {1}
    ORDER BY list_price DESC;
""".format(lbound, ubound)

print(sql_query)


    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN 15.0 AND 20.0
    ORDER BY list_price DESC;



In [9]:
configs = {
    'user': user_id,
    'password': pwd,
    'host': host_name,
    'database': db_name,
    'auth_plugin': 'mysql_native_password',
    'raise_on_warnings': True
}

conn = mysql.connector.connect(**configs)

df = pd.read_sql(sql_query, conn)

conn.close()
df.tail()

C:\Users\Student\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,product_id,product_name,list_price
0,57,Northwind Traders Ravioli,19.50
1,40,Northwind Traders Crab Meat,18.40
2,1,Northwind Traders Chai,18.00
3,66,Northwind Traders Tomato Sauce,17.00
4,86,Northwind Traders Cake Mix,15.99


## 4.0. Using the SQLAlchemy Connection Library

In [10]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"

sqlEngine = create_engine(conn_str, pool_recycle=3600)
conn = sqlEngine.connect()

df = pd.read_sql(sql_query, conn);

conn.close()
df.head()

,product_id,product_name,list_price
0,57,Northwind Traders Ravioli,19.50
1,40,Northwind Traders Crab Meat,18.40
2,1,Northwind Traders Chai,18.00
3,66,Northwind Traders Tomato Sauce,17.00
4,86,Northwind Traders Cake Mix,15.99


## 5.0 Define Helper Functions to Encapsulate and Abstract the Implementation Details

In [11]:
sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    ORDER BY list_price DESC;
"""

### 5.1. Using Individual Connection Parameters

In [12]:
def get_pymysql_dataframe(host, user, password, database_name, sql_query_string):
    connection = pymysql.connect(host=host, user=user, password=password, database=database_name)
    dframe = pd.read_sql(sql_query_string, connection)
    connection.close()
    
    return dframe

In [13]:
df = get_pymysql_dataframe(host_name, user_id, pwd, db_name, sql_query)
df.head()

C:\Users\Student\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,product_id,product_name,list_price
0,20,Northwind Traders Marmalade,81.0
1,51,Northwind Traders Dried Apples,53.0
2,43,Northwind Traders Coffee,46.0
3,8,Northwind Traders Curry Sauce,40.0
4,17,Northwind Traders Fruit Cocktail,39.0


In [14]:
print("Shape: {}\n".format(df.shape))

Shape: (45, 3)



#### 5.1.1 Using SqlAlchemy 

In [16]:
def get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe

In [17]:
df = get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, sql_query)
df.head()

,product_id,product_name,list_price
0,20,Northwind Traders Marmalade,81.0
1,51,Northwind Traders Dried Apples,53.0
2,43,Northwind Traders Coffee,46.0
3,8,Northwind Traders Curry Sauce,40.0
4,17,Northwind Traders Fruit Cocktail,39.0


In [18]:
print(f"Shape: {df.shape[0]} Observations x {df.shape[1]} Features")

Shape: 45 Observations x 3 Features


### 5.2. Using a Configurations Dictionary

In [19]:
def get_mysql_dataframe(sql_query_string, args):
    connection = mysql.connector.connect(**args)
    dframe = pd.read_sql(sql_query_string, connection)
    connection.close()
    
    return dframe

In [20]:
dframe = get_mysql_dataframe(sql_query, configs)
dframe.tail()

C:\Users\Student\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,product_id,product_name,list_price
40,89,Northwind Traders Peaches,1.5
41,94,Northwind Traders Peas,1.5
42,88,Northwind Traders Pears,1.3
43,92,Northwind Traders Green Beans,1.2
44,93,Northwind Traders Corn,1.2


In [21]:
print(f"Shape: {dframe.shape[0]} Observations x {dframe.shape[1]} Features")

Shape: 45 Observations x 3 Features


## 6.0. Writing a Pandas DataFrame to a SQL Database

In [22]:
def insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, table_name):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    df.to_sql(table_name, con=connection, if_exists='replace') #, index_label='product_id');  'append'
    connection.close()

In [23]:
insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, dframe, 'dim_products')

In [25]:
df = get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, 'SELECT * FROM dim_products')
df.head()

,index,product_id,product_name,list_price
0,0,20,Northwind Traders Marmalade,81.0
1,1,51,Northwind Traders Dried Apples,53.0
2,2,43,Northwind Traders Coffee,46.0
3,3,8,Northwind Traders Curry Sauce,40.0
4,4,17,Northwind Traders Fruit Cocktail,39.0


## 7.0. Explore Pandas DataFrames' Capabilities

### 7.1. Display the Data Type of Each Feature 

In [ ]:
sql_query = "SELECT * FROM northwind.products;"

df = get_mysql_dataframe(sql_query, configs)

In [ ]:
df.dtypes